In [1]:
!git clone -q https://github.com/Ryzhtus/relation-extraction-homework
!pip install -q transformers
!pip install -q razdel

     |████████████████████████████████| 1.4MB 13.5MB/s 
     |████████████████████████████████| 890kB 45.3MB/s 
     |████████████████████████████████| 2.9MB 50.0MB/s 


In [2]:
cd /content/relation-extraction-homework

/content/relation-extraction-homework


In [3]:
from relation_extraction.parser import Parser
from relation_extraction.dataset import REDataset

from transformers import BertTokenizer, BertModel
from torch.utils.data import DataLoader

import torch
import torch.nn as nn
import torch.optim as optim

import warnings
warnings.filterwarnings('ignore')

torch.manual_seed(42)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

tokenizer = BertTokenizer.from_pretrained("distilbert-base-multilingual-cased")

Обработка и загрузка данных

In [4]:
train_parser = Parser(tokenizer, 'data/train/*.ann')
train_dataset = REDataset(train_parser, tokenizer)

100%|██████████| 188/188 [00:11<00:00, 15.70it/s]


In [5]:
test_parser = Parser(tokenizer, 'data/test/test_full/*.ann')
test_dataset = REDataset(test_parser, tokenizer)

100%|██████████| 30/30 [00:01<00:00, 18.86it/s]


In [6]:
train_iterator = DataLoader(dataset=train_dataset,
                            batch_size=16,
                            shuffle=True,
                            collate_fn=train_dataset.paddings)

test_iterator = DataLoader(dataset=test_dataset,
                           batch_size=16,
                           shuffle=False,
                           collate_fn=test_dataset.paddings)

Модель

In [7]:
class CustomBert(nn.Module):
    def __init__(self, hidden_dim, num_classes, dropout_level, num_layers=1):
        super().__init__()
        
        self.hidden_dim = hidden_dim
        self.embedding_dim = 768
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.dropout_level = dropout_level
        
        self.bert = BertModel.from_pretrained("distilbert-base-multilingual-cased")

        self.lstm = nn.LSTM(self.embedding_dim, self.hidden_dim, self.num_layers,
                          bidirectional=True, batch_first=True)
        
        self.rnn = nn.RNN(self.hidden_dim * 2, self.hidden_dim)

        self.linear = nn.Linear(self.hidden_dim, self.num_classes)
        
        self.dropout = nn.Dropout(self.dropout_level)
        
    def forward(self, tokens):
        tokens = tokens.permute(1, 0)

        embeddings = self.bert(tokens)[0]
        embeddings_drop = self.dropout(embeddings)

        lstm_out, _ = self.lstm(embeddings_drop)
        lstm_out = lstm_out.permute(1, 0, 2)      
        lstm_drop = self.dropout(lstm_out)
        
        rnn_out, _ = self.rnn(lstm_drop)

        predictions = self.linear(rnn_out)

        return predictions

Обучение

In [9]:
from relation_extraction.train import train_model

classes = 13
hidden = 512
dropout = 0.2

model = CustomBert(hidden, classes, dropout).to(device)

optimizer = optim.Adam(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss(ignore_index=0).to(device)

train_model(model, criterion, optimizer, train_iterator, test_iterator, train_dataset.rel_tag2idx, device, 4)

Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing BertModel: ['distilbert.embeddings.word_embeddings.weight', 'distilbert.embeddings.position_embeddings.weight', 'distilbert.embeddings.LayerNorm.weight', 'distilbert.embeddings.LayerNorm.bias', 'distilbert.transformer.layer.0.attention.q_lin.weight', 'distilbert.transformer.layer.0.attention.q_lin.bias', 'distilbert.transformer.layer.0.attention.k_lin.weight', 'distilbert.transformer.layer.0.attention.k_lin.bias', 'distilbert.transformer.layer.0.attention.v_lin.weight', 'distilbert.transformer.layer.0.attention.v_lin.bias', 'distilbert.transformer.layer.0.attention.out_lin.weight', 'distilbert.transformer.layer.0.attention.out_lin.bias', 'distilbert.transformer.layer.0.sa_layer_norm.weight', 'distilbert.transformer.layer.0.sa_layer_norm.bias', 'distilbert.transformer.layer.0.ffn.lin1.weight', 'distilbert.transformer.layer.0.ffn.lin1.bias', 'distilbert.transformer.layer.0.ffn.lin2.

Epoch 1 / 4
Train Loss = 1.83392, F1-score = 40.126%
Epoch 2 / 4
Train Loss = 1.82487, F1-score = 40.802%
Epoch 3 / 4
Train Loss = 1.82330, F1-score = 41.035%
Epoch 4 / 4
Train Loss = 1.82749, F1-score = 40.787%


Проверка модели на тесте

In [10]:
from relation_extraction.train import eval_epoch

eval_epoch(model, criterion, test_iterator, train_dataset.rel_tag2idx, device)

Test Loss = 2.06200, F1-score = 25.836%
